In [1]:
%load_ext autoreload
%autoreload 2

from src.util import get_path
get_path()

C:/Users/jy/AppData/Local/bipy/31094803/projects/57754191e2d34b7296075e94c545cc4d


In [2]:
try:
    import bql
    import sys

    sys.path.append('./Lib')
    from etiqalib import *
    from etiqalib.config import *
    from etiqalib.bqfields import BaseGetData
    from datetime import datetime

    read = False

    bq = bql.Service()
    def px_function(bq, ticker, start, end):
        bqapi = BaseGetData(bq)
        dff = bqapi.get_data(ticker,{'px_last':bq.data.px_last(start=start, end=end, **fillprev)})
        return dff.drop('CURRENCY',1).dropna(subset=['px_last'])

    start_date = '1998-01-01'
    end_date = datetime.today().date().isoformat()

    if not read:
        opr_df = download_incremental(bq, 'MAOPRATE Index',px_function, start_date, end_date)
        srr_df = download_incremental(bq, 'MAOPSRR Index',px_function, start_date, end_date)
        m3_df = download_incremental(bq, 'MAMS3YOY Index',px_function, start_date, end_date)
        magy10yr = download_incremental(bq, 'MAGY10YR Index',px_function, start_date, end_date)
        price_df = download_incremental(bq, 'FBMKLCI Index', px_function, start_date, end_date)

        opr_df.to_csv('data/opr/opr_rate.csv')
        srr_df.to_csv('data/opr/srr_rate.csv')
        m3_df.to_csv('data/opr/m3_rate.csv')
        magy10yr.to_csv('data/opr/magy10yr.csv')
        price_df.to_csv('data/opr/fbmklci.csv')
        dt = opr_df["DATE"].max()
        print(f"Refreshed data: {dt}")
    else:
        opr_df = pd.read_csv('data/opr/opr_rate.csv', parse_dates=['DATE'])
        srr_df = pd.read_csv('data/opr/srr_rate.csv', parse_dates=['DATE'])
        m3_df = pd.read_csv('data/opr/m3_rate.csv', parse_dates=['DATE'])
        magy10yr = pd.read_csv('data/opr/magy10yr.csv', parse_dates=['DATE'])
        price_df = pd.read_csv('data/opr/fbmklci.csv', parse_dates=['DATE'])
except Exception as err:
    print("Running on local.")
    

    

Running on local.


In [3]:
raise_1 = False
try:
    import bql
    raise_1 = True
except:
    raise_1 = False
if raise_1:
    raise Exception("Query only in BQNT environment.")

In [4]:
from src.jinja_html import render_html, encode_graph
import weasyprint
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', 50) 

import os

In [5]:
def get_year_month(df):  
    df['year_month'] = df['DATE'].map(lambda x: datetime(x.year, x.month, 1).date())
    return df

def filter_date(df,start_date, end_date, date_col='year_month'):
    return df.query('{dtcol} >= "{}" and {dtcol} <= "{}"'.format(start_date, end_date, dtcol=date_col))

def read_csv(file):
    '''
    with parse_date
    '''
    return pd.read_csv(file, parse_dates=['DATE'])

# os.listdir('data/Loans Stats/')

### Parameters

In [6]:
image_folder = 'output'

# dt = datetime(2022,8,1)
dt = datetime.now()

dt_mth = dt.strftime("%Y%m")
file_1 = f'data/Loans Stats/{dt_mth}_Banking System.XLS'
file_2 = f'data/Loans Stats/{dt_mth}_Banking System_2.XLS'

start_date = "2010-01-01"

opr_df         = read_csv('data/opr/opr_rate.csv')
end_date = opr_df["DATE"].max() - pd.offsets.BMonthEnd(1)
# end_date = "2022-02-28"  # to update
print("End Date:", end_date)
print(dt_mth, file_1, file_2)

End Date: 2022-09-30 00:00:00
202210 data/Loans Stats/202210_Banking System.XLS data/Loans Stats/202210_Banking System_2.XLS


### Read Data

In [7]:
loan_3_df        = pd.read_excel(file_1, sheet_name='Loan_3')
int_df           = pd.read_excel(file_1, sheet_name='Int')
npl_df           = pd.read_excel(file_1, sheet_name='NPL')[['Unnamed: 0','Unnamed: 17']]
application_df = pd.read_excel(file_2, sheet_name='Applications by purpose', header=None)
approval_df    = pd.read_excel(file_2, sheet_name='Approvals by purpose', header=None)


opr_df         = read_csv('data/opr/opr_rate.csv').pipe(get_year_month).pipe(filter_date, start_date, end_date, "DATE")
m3_price       = read_csv('data/opr/m3_rate.csv').pipe(get_year_month).pipe(filter_date, start_date, end_date, "DATE") 
srr            = read_csv('data/opr/srr_rate.csv').pipe(get_year_month).pipe(filter_date, start_date, end_date, "DATE")
bond_yields    = read_csv('data/opr/magy10yr.csv').pipe(filter_date, start_date, end_date, "DATE")

In [8]:
# !pip install xlrd --upgrade

### Data Processing

In [9]:
def trim_df(df):
    dff = df[list(range(15))].copy()
    dff.columns  = dff.iloc[3][:2].append(dff.iloc[2][2:]).tolist()
    dff = dff.iloc[4:].copy().dropna(subset=['Month'])
    dff['Year'] = dff['Year'].ffill()
    dff['year_month'] = dff.apply(lambda x: '{}-{}-01'.format(x['Year'], str(x['Month']).zfill(2)),1)
    return dff

def process_npl(npl):
    npl.columns= ['Date', 'Gross NPL']
    npl = npl.iloc[npl[npl['Date']=='Monthly'].index[0]+1:].dropna().copy()
    npl['Date'] = pd.to_datetime(npl['Date'])
    return npl

def process_opr(opr_df):
    def label(x):
        if x > 0:
            return 'hike'
        elif x< 0 :
            return 'cut'
        else:
            return 'stay'
    
    opr_df['label'] = opr_df['px_last'].diff().map(label) 
    opr_month = pd.pivot_table(opr_df, index='year_month', columns='label', values='px_last').fillna(0)\
                  .applymap(lambda x: 1 if x!= 0 else 0).reset_index()
    opr_month['label'] = opr_month.apply(lambda x: 'cut' if x['cut']==1 else ('hike' if x['hike'] == 1 else 'stay'), 1)
    return opr_df, opr_month

application_df_1 = trim_df(application_df).pipe(filter_date, start_date, end_date)
approval_df_1 = trim_df(approval_df).pipe(filter_date, start_date, end_date).\
                                    rename(columns={'Residential':'Residential mortgages',
                                                    'Non-residential':'Non-residential mortgages',
                                                     'Consumer goods':'Consumer durable goods', 
                                                    'Other ':'Other purpose'})
npl = process_npl(npl_df).pipe(filter_date, start_date, end_date, "Date")
opr_df, opr_month = process_opr(opr_df)

m3_price_m = m3_price.groupby(['ID','year_month'])['px_last'].last().reset_index()
srr_m = srr.groupby(['ID','year_month'])['px_last'].last().reset_index()

In [50]:
application_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110
0,Banking System: Loans Applied by Purpose,NaN,NaN,NaN,NaN,NaN,NaN,156,156,78,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,YoY growth,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Applications (yoy % growth),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Purpose (RM m),NaN,Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL,NaT,NaN,NaN,Purpose (RM m),NaN,Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,...,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL,NaN,Purpose (RM m),Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL
3,Year,Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Year,Month,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,1,1495.652,3850.558,3771.661,3357.44,1252.459,429.307,670.486,1193.565,0.588,248.005,4934.69,2007.261,19440.011,2006-01-31,19440.011,NaN,2006,1,NaN,NaN,NaN,NaN,NaN,...,53356.639,23748.871,5398.928,10167.504,21592.833,116.874,9685.911,87663.386,29605.41,300843.147,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,NaN,NaN,NaN,NaN,NaN,0.40348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,NaN,NaN,NaN,NaN,NaN,0.086357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82284.321,NaT,396469.539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
application_df_1.tail(13)

,Year,Month,Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL,year_month
Date,,,,,,,,,,,,,,,,
2021-02-01,2021,2,2652.845,5868.178,5529.921,20538.602,5716.006,584.714,5126.201,2241.015,0.031,3104.373,11226.781,1948.439,59007.185,2021-02-01
2021-03-01,2021,3,4035.132,10772.47,10304.096,36142.102,8707.618,1623.959,6371.239,2996.954,45.039,3970.243,16745.036,2827.631,94237.423,2021-03-01
2021-04-01,2021,4,3004.441,10905.079,10461.235,36740.881,9430.79,1416.722,6017.375,2934.494,0.028,4237.64,16701.969,2218.936,93608.355,2021-04-01
2021-05-01,2021,5,2653.941,7592.894,7138.037,33010.433,7941.362,1297.48,4871.416,2618.381,0,3578.152,14175.752,1885.973,79625.784,2021-05-01
2021-06-01,2021,6,4585.227,3890.262,3578.428,27101.597,7244.168,1315.59,4859.217,2618.695,0.011,2274.179,14940.596,1122.225,69951.767,2021-06-01
2022-01-01,2022,1,3798.93971,12701.675598,12190.008961,38226.207668,8565.824657,1851.40592,5949.120601,3793.613657,8.542845,2528.548143,15426.683114,5074.884251,97925.446164,2022-01-01
2022-02-01,2022,2,2970.525565,10356.763363,9986.890104,28214.787496,7356.492359,862.297781,5151.648896,3301.010512,8.573237,2707.99838,17725.780573,3165.85981,81821.737973,2022-02-01
2022-03-01,2022,3,3895.345478,14161.529183,13517.019565,42937.240571,10813.843221,1120.887426,6817.964151,3862.41312,8.718914,3739.024911,28757.563923,3971.172198,120085.703096,2022-03-01
2022-04-01,2022,4,2963.680764,13997.126546,13380.819148,40754.531817,11147.560153,1170.434051,5693.455744,3975.155244,8.077268,5238.190366,28391.3751,3848.215981,117187.803036,2022-04-01


In [10]:
# end_date

In [11]:
loan_last_date = npl["Date"].values[-1]
print(loan_last_date)

import datetime
loan_last_date = pd.to_datetime("2022-08-01") # override (missing august data for october xlsx)
loan_last_date

2022-07-01T00:00:00.000000000


Timestamp('2022-08-01 00:00:00')

In [47]:
application_df_1.index[-13:]

DatetimeIndex(['2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01',
               '2021-06-01', '2022-01-01', '2022-02-01', '2022-03-01',
               '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01',
               '2022-08-01'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [45]:
application_df_1.loc["2021-07":]

,Year,Month,Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL,year_month
Date,,,,,,,,,,,,,,,,
2022-01-01,2022,1,3798.93971,12701.675598,12190.008961,38226.207668,8565.824657,1851.40592,5949.120601,3793.613657,8.542845,2528.548143,15426.683114,5074.884251,97925.446164,2022-01-01
2022-02-01,2022,2,2970.525565,10356.763363,9986.890104,28214.787496,7356.492359,862.297781,5151.648896,3301.010512,8.573237,2707.99838,17725.780573,3165.85981,81821.737973,2022-02-01
2022-03-01,2022,3,3895.345478,14161.529183,13517.019565,42937.240571,10813.843221,1120.887426,6817.964151,3862.41312,8.718914,3739.024911,28757.563923,3971.172198,120085.703096,2022-03-01
2022-04-01,2022,4,2963.680764,13997.126546,13380.819148,40754.531817,11147.560153,1170.434051,5693.455744,3975.155244,8.077268,5238.190366,28391.3751,3848.215981,117187.803036,2022-04-01
2022-05-01,2022,5,2710.778105,12747.343584,12242.661805,38931.925161,9191.797933,852.554147,5368.080565,3661.082145,8.926248,3775.820908,20790.160882,2462.488624,100500.958302,2022-05-01
2022-06-01,2022,6,5555.187349,14852.133768,14193.207727,46676.019675,11671.266702,1525.650843,6898.391349,3872.688123,9.687413,6860.630226,24978.432594,3078.629803,125978.717845,2022-06-01
2022-07-01,2022,7,3112.193473,14107.354983,13518.937919,39645.060998,11415.65656,1147.219873,6681.62645,4131.039816,14.801454,13727.832926,33994.598246,3389.180875,131366.565654,2022-07-01
2022-08-01,2022,8,2985.741109,15144.70366,14477.138666,41807.96102,11563.254548,1058.086652,7341.319516,4856.37456,18.575982,11470.791507,27283.283944,5471.411239,129001.503736,2022-08-01


### Visualization

In [12]:
width = 1200
height = 625

def add_title(fig, title, ylabel="Amount (billion) [MYR]"):
    fig.update_layout(template='ygridoff',
                      title=title,
                    yaxis=dict(
                    title=ylabel,
                ),
                )
    return fig


def plot_multi_lines(df, x_axis, y_axes):
    fig = go.Figure()
    for col in y_axes:
        fig.add_trace(go.Scatter(x=df[x_axis], y=df[col]/1000, name=col))
    fig.update_xaxes(dtick='M12')
    return fig


### Overview of Bank Loan Stats

In [13]:
def create_overview_fig(m3_price_m, srr_m, npl, opr_df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = npl['Date'],
                             y = npl['Gross NPL'],
                             marker_color='#7f7f7f',
                             name = 'NPL',
                            yaxis="y2"))

    fig.add_trace(go.Scatter(x = m3_price_m['year_month'],
                             y = m3_price_m['px_last'],
                             marker_color='#ff7f0e',
                             name = 'M3'),
                 )

    fig.add_trace(go.Scatter(x = srr_m['year_month'],
                             y = srr_m['px_last'],
                             marker_color='red',
                             name = 'SRR'),
                 )


    fig.add_trace(go.Scatter(x = opr_df['DATE'],
                             y = opr_df['px_last'],
                             marker_color='blue',
                             mode='lines',
                             name = 'OPR',
                             yaxis="y2"),
                 )

    fig.update_layout(
        title='NPL',
        yaxis=dict(
            title="M3 / SRR",
            titlefont=dict(
                color="red"
            ),
            tickfont=dict(
                color="red"
            )
        ),
        yaxis2=dict(
            title="Gross NPL / OPR",
            anchor="x",
            overlaying="y",
            side="right",
            position=0.85
        )
    )

    fig.update_layout(
        template='ygridoff',
        xaxis = dict(
            dtick = "M12"
        )
    )
    fig.update_layout(
        autosize=False,
        width=width,
        height=height,)
    return fig


In [14]:
fig_overview = create_overview_fig(m3_price_m, srr_m, npl, opr_df)


In [15]:
x1 = m3_price_m.tail(13)
d1, d2 = x1.year_month.values[0], x1.year_month.values[-1]
m3_1, m3_2 = x1.loc[:, "px_last"].values[0], x1.loc[:, "px_last"].values[-1]

x1 = npl.tail(13)
d1, d2 = x1.Date.values[0], x1.Date.values[-1]
npl_1, npl_2 = x1.loc[:, "Gross NPL"].values[0], x1.loc[:, "Gross NPL"].values[-1]

if npl_2 >= npl_1:
    compare = "higher"
else:
    compare = "lower"
overview_text = "M3 YoY Growth is {:.2f}%. Non-performing Loan (NPL) is {:.2f}%, which is {} compared to last year ({:.2f}%)".format(m3_2, npl_2, compare, npl_1)



### Total Bank Loan Application and Approval

In [35]:
print(loan_last_date)
application_df_1["Date"] = pd.to_datetime(application_df_1["year_month"])
application_df_1 = application_df_1.set_index("Date").sort_index().loc[:loan_last_date]
approval_df_1["Date"] = pd.to_datetime(approval_df_1["year_month"])
approval_df_1 = approval_df_1.set_index("Date").sort_index().loc[:loan_last_date]
application_df_1

2022-08-01 00:00:00


,Year,Month,Purchase of securities,Auto,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Consumer durable goods,Construction,Working capital,Other purpose,TOTAL,year_month
Date,,,,,,,,,,,,,,,,
2010-01-01,2010,1,2014.997,6716.232,6223.617,12039.374,5215.867,507.894,2127.365,1776.912,11.157,3600.621,10855.195,2967.164,47832.778,2010-01-01
2010-02-01,2010,2,1431.603,4408.83,4150.592,8337.41,3752.318,289.203,1629.286,1316.803,0.889,2262.172,6583.449,3503.173,33515.136,2010-02-01
2010-03-01,2010,3,1677.771,7565.46,7180.782,14907.987,6855.434,535.581,2718.741,1824.71,1.053,2347.518,10012.144,3322.06,51768.459,2010-03-01
2010-04-01,2010,4,1861.384,6071.498,5639.497,15156.283,7366.605,397.593,3029.363,2224.193,1.865,2059.239,12285.032,2994.56,53447.615,2010-04-01
2010-05-01,2010,5,2885.226,6754.213,6280.837,13844.545,6096.749,625.666,3336.039,2160.483,0.843,2890.233,11035.663,2836.146,52465.806,2010-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,2022,4,2963.680764,13997.126546,13380.819148,40754.531817,11147.560153,1170.434051,5693.455744,3975.155244,8.077268,5238.190366,28391.3751,3848.215981,117187.803036,2022-04-01
2022-05-01,2022,5,2710.778105,12747.343584,12242.661805,38931.925161,9191.797933,852.554147,5368.080565,3661.082145,8.926248,3775.820908,20790.160882,2462.488624,100500.958302,2022-05-01
2022-06-01,2022,6,5555.187349,14852.133768,14193.207727,46676.019675,11671.266702,1525.650843,6898.391349,3872.688123,9.687413,6860.630226,24978.432594,3078.629803,125978.717845,2022-06-01


In [40]:
approval_df_1["TOTAL"].pct_change(12)

Date
2010-01-01         NaN
2010-02-01         NaN
2010-03-01         NaN
2010-04-01         NaN
2010-05-01         NaN
                ...   
2022-04-01    0.662651
2022-05-01    0.646428
2022-06-01    1.267142
2022-07-01    0.776529
2022-08-01    0.814255
Name: TOTAL, Length: 152, dtype: float64

In [17]:
def create_loan_app_fig(application_df_1, approval_df_1, opr_df):
    fig2 = go.Figure()
    
    fig2.add_trace(go.Scatter(x = application_df_1.dropna()['year_month'],
                             y = application_df_1.dropna()['TOTAL']/1000,
                             marker_color='#ffcfcf',
                             name = 'Application'))
        
    fig2.add_trace(go.Scatter(x = application_df_1.dropna()['year_month'],
                             y = application_df_1.dropna()['TOTAL'].rolling(12).mean()/1000,
                             marker_color='red',
                             name = 'Application (Avg)'))
    
    fig2.add_trace(go.Scatter(x = approval_df_1.dropna()['year_month'],
                             y = approval_df_1.dropna()['TOTAL']/1000,
                             marker_color='#b6fac1',
                            name = 'Approval'),
                 )
    
    fig2.add_trace(go.Scatter(x = approval_df_1.dropna()['year_month'],
                             y = approval_df_1.dropna()['TOTAL'].rolling(12).mean()/1000,
                             marker_color='green',
                            name = 'Approval (Avg)'),
                 )

    fig2.add_trace(go.Scatter(x = opr_df.query('DATE >= "2006-01-01"')['DATE'],
                             y = opr_df.query('DATE >= "2006-01-01"')['px_last'],
                             marker_color='blue',
                             mode='lines',
                             name = 'OPR',
                            yaxis="y2"),
                 )

    fig2.update_layout(
        title='Total Bank Loan Application and Approval (Rolling 12 Months Average)',
        yaxis=dict(
            title="Application / Approval  (billion) [MYR]",
            titlefont=dict(
                color="#7f7f7f"
            ),
            tickfont=dict(
                color="#7f7f7f"
            )
        ),
        yaxis2=dict(
            title="OPR",
            titlefont=dict(
                color="blue"
            ),
            tickfont=dict(
                color="blue"
            ),
            anchor="x",
            overlaying="y",
            side="right",
            position=0.85
        )
    )

    fig2.update_layout(
        template='ygridoff',
        xaxis = dict(
            dtick = "M12"
        )
    )
    fig2.update_layout(
        autosize=False,
        width=width,
        height=height,)
    return fig2

In [18]:
fig_loan = create_loan_app_fig(application_df_1, approval_df_1, opr_df)


In [19]:
x1 = application_df_1.tail(13)
appl_1, appl_2, appl_3 = x1.loc[:, "TOTAL"].values[0], x1.loc[:, "TOTAL"].values[-2], x1.loc[:, "TOTAL"].values[-1]
appl_yoy = appl_3/appl_1-1
appl_mom = appl_3/appl_2-1

x1 = approval_df_1.tail(13)
appr_1, appr_2, appr_3 = x1.loc[:, "TOTAL"].values[0], x1.loc[:, "TOTAL"].values[-2], x1.loc[:, "TOTAL"].values[-1]
appr_yoy = appr_3/appr_1-1
appr_mom = appr_3/appr_2-1

latest_month = x1.index[-1].strftime("%B %Y")
loan_text = "As of {}, Total Bank Loan Application MoM and YoY change are {:.2f}% and {:.2f}% respectively;".format(latest_month, appl_mom*100, appl_yoy*100)
loan_text += " Total Bank Loan Approval MoM and YoY change are {:.2f}% and {:.2f}% respectively.".format(appr_mom*100, appr_yoy*100)


In [20]:
appl_1, appl_2, appl_3

(59007.185000000005, 131366.56565415, 129001.50373577993)

In [21]:

application_df_1["Date"] = pd.to_datetime(application_df_1["year_month"])
purposes = [i for i in application_df_1 if i not in ['Date', 'Year','Month','TOTAL','year_month','Auto','Consumer durable goods']]
application_df_1a = application_df_1.set_index("Date")[purposes]
application_df_2 = application_df_1a.rolling(12).mean()
approval_df_1["Date"] = pd.to_datetime(approval_df_1["year_month"])
approval_df_1a = approval_df_1.set_index("Date")[purposes]
approval_df_2 = approval_df_1a.rolling(12).mean()


In [22]:
approval_df_2

,Purchase of securities,Passenger cars,Residential mortgages,Non-residential mortgages,Purchase of fixed assets,Personal uses,Credit cards,Construction,Working capital,Other purpose
Date,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-01,2227.993047,5701.387369,13215.190059,4060.278807,762.107262,1888.995116,1110.289169,1733.108578,12054.652211,2206.335077
2022-05-01,2268.831469,5895.408916,13566.754409,4219.116997,845.992223,1897.906508,1169.447325,1838.053756,12541.763845,2461.007991
2022-06-01,2312.520712,6467.544543,14268.486906,4510.902451,791.658048,1950.149050,1238.974862,2054.315393,13536.643912,2743.725446


### Loan App & Approval

In [23]:
fig_application = plot_multi_lines(application_df_2.reset_index(), 'Date', purposes)
fig_application = add_title(fig_application, 'Application by Purpose', ylabel = "Amount (billion MYR)")
fig_application = fig_application.update_layout(
    autosize=False,
    width=width,
    height=height,)

fig_approval = plot_multi_lines(approval_df_2.reset_index(), 'Date', purposes)
fig_approval = add_title(fig_approval, 'Approval by Purpose', ylabel = "Amount (billion MYR)")
fig_approval = fig_approval.update_layout(
    autosize=False,
    width=width,
    height=height,)



In [24]:
# a1 = approval_df_2.loc["2014", ["Residential mortgages"]]
# a2 = approval_df_2.loc["2022", ["Residential mortgages"]]
# pd.concat([a1, a2])

In [25]:
# application_df_1["Date"] = pd.to_datetime(application_df_1["year_month"])
# application_df_1a = application_df_1[purposes].set_index("Date")
# application_df_1a

In [26]:
def preproc_app_df(df):
    app_df_1 = df.pct_change(12).loc[df.index[-4]:]
    col_1 = app_df_1.T.columns[-1]
    new_col = [j.strftime("%b%y-YoY") for j in app_df_1.T.columns]
    table_application_1 = (  app_df_1.T.sort_values(col_1, ascending=False)*100).round(2)
    table_application_1.columns = new_col
    return table_application_1

def create_styled_table(app_df):
    app_df_1 = app_df.copy()
    app_df_1.index.name = "Purpose"
    app_df_2 = app_df_1.reset_index()
    idxs = app_df_2.index[app_df_2["Purpose"].isin(["Working capital", "Credit cards", "Residential mortgages", "Passenger cars"])]

    fdf_styled = app_df_2.style.set_properties(**{
        "text-align": "right",
        'font-size': '8pt',
        'white-space': 'pre-wrap',
    })
    cols = fdf_styled.columns
    bd_color = "#acacad"
    props = {
        col: {"padding": "3px",  "text-align": "right", 'border-right': f'2px solid {bd_color}'}
        for col in cols
    }
    fdf_styled = fdf_styled.format(
        precision=2, subset=cols)

    for key, val in props.items():
        fdf_styled = fdf_styled.set_properties(subset=[key], **val)

    fdf_styled = fdf_styled.set_table_styles(
        [
            {
                "selector": ".row_heading, .blank", "props": [
                    ("display", "none;"),
                    # ('border-bottom', '2px solid #e3987b')
                ]
            },
            {'selector': 'th', 'props': [
                ('font-size', '8pt'),
                ('text-align', 'right'),
                ('max-width', '35px'),
                ('border-bottom', '3px solid #acacad')
            ]}
        ]
    )
    idx = pd.IndexSlice
    fdf_styled = fdf_styled.set_properties(**{"background-color": "#d4e1ff"}, subset=idx[idxs,:])
    return fdf_styled


In [27]:
appl_df_1 = preproc_app_df(application_df_1a)
appr_df_1 = preproc_app_df(approval_df_1a)
appl_df_2 = create_styled_table(appl_df_1)
appr_df_2 = create_styled_table(appr_df_1)

---
### Top 3 sectors

In [28]:
# lowest = application_df_1.query('Year == 2020').query('Month == 4').index[0]
# latest = application_df_1.index[-1]
# window = int(latest-lowest)
# print(window, type(window))

# top_3_recoveries = application_df_1[purposes].diff(window).iloc[-1].sort_values(ascending=False)[:3].index
# top_3_recoveries

In [29]:
def plot_multi_lines_2(df, x_axis, y_axes):
    colors = ["rgb(252, 32, 3)", "rgb(255, 172, 161, 0.5)", "rgb(0, 176, 88)", "rgb(149, 240, 194, 0.5)"]
    opacitys = [1, 0.5, 1, 0.5]
    fig = go.Figure()
    for col, color, opacity in zip(y_axes, colors, opacitys):
        fig.add_trace(go.Scatter(x=df[x_axis], y=df[col]/1000, name=col, 
                                 line=dict(
                                     # opacity=opacity,
                                     color=color
                                 )
                                )
                     )
    fig.update_xaxes(dtick='M12')
    return fig

def appl_vs_approv(col, smooth=True):
    p_df = application_df_1[['year_month',col]].rename(columns={col:'Application'})\
                        .merge(approval_df_1[['year_month',col]].rename(columns={col:'Approval'})).copy()
    if smooth:
        p_df['Application (Avg)'] = p_df['Application'].rolling(12).mean()
        p_df['Approval (Avg)'] = p_df['Approval'].rolling(12).mean()
    fig = plot_multi_lines_2(p_df, 'year_month', ['Application (Avg)', 'Application','Approval (Avg)', 'Approval'])
    fig = add_title(fig, col + " (Rolling 12 Months Average)")
    return fig

def create_loan_fig(col, dts_1, dts_2):
    fig6a = appl_vs_approv(col)

    fillcolor = "#a7becf"
    layer = "below"
    dt1, dt2 = dts_1
    dt1b, dt2b = dts_2
    fig6a.add_shape(type="rect",
                            xref="x",
                            yref="paper",
                            x0=dt1,
                            y0=0,
                            x1=dt2,
                            y1=1,
                            line=dict(color="rgba(0,0,0,0)",width=3,),
                            fillcolor=fillcolor,
                            layer=layer) 
    fig6a.add_shape(type="rect",
                            xref="x",
                            yref="paper",
                            x0=dt1b,
                            y0=0,
                            x1=dt2b,
                            y1=1,
                            line=dict(color="rgba(0,0,0,0)",width=3,),
                            fillcolor=fillcolor,
                            layer=layer) 
    fig = fig6a.update_layout(
        autosize=False,
        width=width,
        height=height,)
    return fig


In [30]:
app_cols = ["Passenger cars", "Residential mortgages", "Credit cards", "Working capital"]
app_cols_2 = ["Passenger Cars", "Residential Mortgages", "Credit cards", "Working Capital"]

dates_1 = [("2013-01-01", "2013-12-31"), ("2013-10-01", "2014-12-31"), ("2017-01-01", "2018-12-31"), ("2015-12-01", "2016-09-30")]
dates_2 = [("2021-01-01", "2022-01-31"), ("2021-01-01", "2022-01-31"), ("2021-01-01", "2022-01-31"), ("2020-02-01", "2020-12-31")]

In [31]:
figs = []
for col, dts_1, dts_2 in zip(app_cols, dates_1, dates_2):
    figs.append(create_loan_fig(col, dts_1, dts_2))

In [32]:
purpose_texts = []
for col in app_cols:
    v1 = appl_df_1.loc[col, :][-1]
    v2 = appr_df_1.loc[col, :][-1]
    txt = "{} Application YoY Change is {:.2f}%, Approval YoY Change is {:.2f}%".format(col, v1, v2)
    purpose_texts.append(txt)


### Generate PDF

In [33]:
%%time
# dt = pd.to_datetime(loan_last_date)

date_filename = dt.strftime("%Y-%m-%d")
date_time = dt.strftime("%B %Y")
title = "Bank Loan Stats"
data = {
    "title": title,
    "date_time_str": f"{date_time} Update",
    "fig_overview": encode_graph(fig_overview),
    "overview_text": overview_text,
    "fig_loan": encode_graph(fig_loan),
    "loan_text": loan_text,
    "fig_application": encode_graph(fig_application),
    "fig_approval": encode_graph(fig_approval),
    "appl_df_2": appl_df_2.to_html(),
    "appr_df_2": appr_df_2.to_html(),
    "fig_titles": app_cols_2,
    "figs": [encode_graph(fig) for fig in figs],
    "purpose_texts": purpose_texts, 
    "figs_N": len(figs),
}

output_html_filename = f"output/{title}-{date_filename}.html"
output_pdf_filename = f"output/{title}-{date_filename}.pdf"
render_html(data, output_filename=output_html_filename,
            input_template_filename="bank_loan_stats_template.html")
os.add_dll_directory(r"C:\Program Files\GTK3-Runtime Win64\bin")
doc_1 = weasyprint.HTML(filename=output_html_filename)
doc_pdf = doc_1.write_pdf(output_pdf_filename)


Wall time: 4.41 s


In [34]:
print("Done output:", output_pdf_filename)

Done output: output/Bank Loan Stats-2022-10-03.pdf
